# Activity 8

This activity tests your skills
in formulating an engineering problem as solving a system of linear algebraic equations,
as well as analyzing results thereof.

## Scenario

The lateral view of an [inclined strut](./act-08.png)
is designed as a ten-member truss.

You are tasked to analyze a scenario wherein
loads are approximated as concentrated forces 
acting on the non-support joints.
Members $AC$, $CE$, and $EG$ are collinear,
and perpendicular to the lines of action of loads $p_{1}$, $p_{2}$, and $p_{3}$.
Members $BD$, $DF$, and $FG$ are collinear,
and perpendicular
not just to the lines of action of loads $p_{4}$ and $p_{5}$,
but also to the supporting surface.

Let the dimensions, material makeup, and weight of the members be negligible.
Assume that the joints do not resist moments.
Determine the member forces developed
under such a conservative idealization of the strut.

## Modelling

Cast the original task as solving a system of linear algebraic equations
for the forces in $AC$, $BC$, $BD$, $CD$, $CE$, $DE$, $DF$, $EF$, $EG$, and $FG$.

Arrange the equations in the matrix-vector form
$\boldsymbol{K} \boldsymbol{\phi} = \boldsymbol{\delta}$,
where
$\boldsymbol{\phi}$ collects the member forces in the order presented above.
For uniformity, assume that tensile forces are positive.

## Core code

In [1]:
import math as mt

import numpy as np

Define a Python function
`get_coeffs_rhs()`
that takes four positional arguments
(namely, `d`, `h`, `theta`, `p1`, `p2`, `p3`, `p4`, `p5`,
in that order)
and returns two NumPy arrays
(namely,
`coeff_matrix`, `rhs_vector`
in that order).
The roles of these inputs and outputs should be evident.

In [2]:
def get_coeffs_rhs(d, h, theta, p1, p2, p3, p4, p5):
    nodes = {
        'B': np.array([0.0, 0.0]),
        'D': np.array([d, 0.0]),
        'F': np.array([2.0*d, 0.0]),
        'G': np.array([3.0*d, 0.0]),
        'A': np.array([0.0, h]),
        'C': np.array([0.5*d, h * (1.0 - 0.5/3.0)]),
        'E': np.array([1.5*d, h * (1.0 - 1.5/3.0)])
    }
    
    member_map = [
        ('A', 'C'), ('B', 'C'), ('B', 'D'), 
        ('C', 'D'), ('C', 'E'), ('D', 'E'), 
        ('D', 'F'), ('E', 'F'), ('E', 'G'), 
        ('F', 'G')
    ]
    
    solve_joints = ['C', 'D', 'E', 'F', 'G']
    coeff_matrix = np.zeros((10, 10))
    rhs_vector = np.zeros(10)
    
    for row_i, joint in enumerate(solve_joints):
        pos_joint = nodes[joint]
        r_x, r_y = 2 * row_i, 2 * row_i + 1
        
        for col_j, (n1, n2) in enumerate(member_map):
            if joint == n1 or joint == n2:
                other = n2 if joint == n1 else n1
                vec = nodes[other] - pos_joint
                u = vec / np.linalg.norm(vec)
                coeff_matrix[r_x, col_j] = u[0]
                coeff_matrix[r_y, col_j] = u[1]
                
    hyp = np.sqrt(h**2 + (3*d)**2)
    u_top = np.array([-h/hyp, -3*d/hyp])
    u_bot = np.array([0.0, -1.0])
    
    load_map = {
        'C': p1 * u_top, 'E': p2 * u_top, 'G': p3 * u_top,
        'F': p4 * u_bot, 'D': p5 * u_bot
    }
    
    for row_i, joint in enumerate(solve_joints):
        if joint in load_map:
            load = load_map[joint]
            rhs_vector[r_x := 2*row_i] -= load[0]
            rhs_vector[r_x + 1] -= load[1]
            
    return coeff_matrix, rhs_vector

## Assessment

Define Python variables
`D`, `H`, `THETA`, `P1`, `P2`, `P3`, `P4`, and `P5`
for storing the values of parameters
$d$, $h$, $\theta$, $p_{1}$, $p_{2}$, $p_{3}$, $p_{4}$, and $p_{5}$,
respectively.

Every time you solve for $\boldsymbol{\phi}$,
- use `get_coeffs_rhs()` to generate the coefficient matrix and the right-hand-side vector,
  respectively storing them in Python variables `K` and `d`;
- show the determinant and the condition number of the coefficient matrix;
  and
- print $\boldsymbol{\phi}$.


### Task 1

Determine the member forces when $C$ is loaded by a unit force and all other joints are unloaded.

Define a Python variable `phi_1` to store $\boldsymbol{\phi}$ for this task.

In [3]:
D, H, THETA = 2.0, 3.0, 17.0
P1, P2, P3, P4, P5 = 9.0, 6.0, 5.0, 1.0, 7.0

def solve_task(loads):
    K, d = get_coeffs_rhs(D, H, THETA, *loads)
    phi = np.linalg.solve(K, d)
    print(f"Determinant:     {np.linalg.det(K):.4f}")
    print(f"Condition Num:   {np.linalg.cond(K):.4f}")
    print("Member Forces (kN):")
    print(phi)
    return phi

phi_1 = solve_task((1, 0, 0, 0, 0))

Determinant:     0.3416
Condition Num:   12.3327
Member Forces (kN):
[-0.08333333 -1.00346621 -0.          0.         -0.         -0.
 -0.          0.         -0.         -0.        ]


### Task 2

Determine the member forces when $E$ is loaded by a unit force and all other joints are unloaded.

Define a Python variable `phi_2` to store $\boldsymbol{\phi}$ for this task.

In [4]:
phi_2 = solve_task((0, 1, 0, 0, 0))

Determinant:     0.3416
Condition Num:   12.3327
Member Forces (kN):
[ 0.75       -0.60207973 -0.89442719  0.90311959  0.125      -1.00778222
 -0.          0.         -0.         -0.        ]


### Task 3

Determine the member forces when $G$ is loaded by a unit force and all other joints are unloaded.

Define a Python variable `phi_3` to store $\boldsymbol{\phi}$ for this task.

In [5]:
phi_3 = solve_task((0, 0, 1, 0, 0))

Determinant:     0.3416
Condition Num:   12.3327
Member Forces (kN):
[ 2.         -0.         -2.23606798  0.          2.         -0.
 -2.23606798  0.          2.         -2.23606798]


### Task 4

Determine the member forces when $F$ is loaded by a unit force and all other joints are unloaded.

Define a Python variable `phi_4` to store $\boldsymbol{\phi}$ for this task.

In [6]:
phi_4 = solve_task((0, 0, 0, 1, 0))

Determinant:     0.3416
Condition Num:   12.3327
Member Forces (kN):
[ 1.49071198 -0.35901099 -1.2         0.53851648  1.11803399 -0.60092521
 -0.66666667  1.20185043 -0.         -0.        ]


### Task 5

Determine the member forces when $D$ is loaded by a unit force and all other joints are unloaded.

Define a Python variable `phi_5` to store $\boldsymbol{\phi}$ for this task.

In [7]:
phi_5 = solve_task((0, 0, 0, 0, 1))

Determinant:     0.3416
Condition Num:   12.3327
Member Forces (kN):
[ 0.74535599 -0.71802197 -0.4         1.07703296 -0.         -0.
 -0.          0.         -0.         -0.        ]


### Task 6

Determine the member forces
when $C$, $E$, $G$, $F$, and $D$
are loaded by unit forces.

Define a Python variable `phi_6` to store $\boldsymbol{\phi}$ for this task.

In [8]:
phi_6 = solve_task((1, 1, 1, 1, 1))

Determinant:     0.3416
Condition Num:   12.3327
Member Forces (kN):
[ 4.90273464 -2.68257891 -4.73049517  2.51866904  3.24303399 -1.60870743
 -2.90273464  1.20185043  2.         -2.23606798]


### Task 7

Similar to Task 6,
determine the member forces
when $C$, $E$, $G$, $F$, and $D$
are loaded by unit forces,
but use the results in Tasks 1 - 5.

Define a Python variable `phi_7` to store $\boldsymbol{\phi}$ for this task.

In [9]:
phi_7 = phi_1 + phi_2 + phi_3 + phi_4 + phi_5
print("Member Forces (kN):")
print(phi_7)

Member Forces (kN):
[ 4.90273464 -2.68257891 -4.73049517  2.51866904  3.24303399 -1.60870743
 -2.90273464  1.20185043  2.         -2.23606798]


### Task 8

Determine the member forces using the assigned values of
$p_{1}$, $p_{2}$, $p_{3}$, $p_{4}$, and $p_{5}$.

Define a Python variable `phi_8` to store $\boldsymbol{\phi}$ for this task.

In [10]:
phi_8 = solve_task((P1, P2, P3, P4, P5))

Determinant:     0.3416
Condition Num:   12.3327
Member Forces (kN):
[ 20.45820393 -18.02883911 -20.54690303  13.49646477  11.86803399
  -6.64761852 -11.84700655   1.20185043  10.         -11.18033989]


### Task 9

Similar to Task 8,
determine the member forces using the assigned values of
$p_{1}$, $p_{2}$, $p_{3}$, $p_{4}$, and $p_{5}$,
but use the results in Tasks 1 - 5.


Define a Python variable `phi_9` to store $\boldsymbol{\phi}$ for this task.

In [11]:
phi_9 = (P1 * phi_1) + (P2 * phi_2) + (P3 * phi_3) + (P4 * phi_4) + (P5 * phi_5)
print("Member Forces (kN):")
print(phi_9)

Member Forces (kN):
[ 20.45820393 -18.02883911 -20.54690303  13.49646477  11.86803399
  -6.64761852 -11.84700655   1.20185043  10.         -11.18033989]


### Task 10

Determine the magnitudes of the support reactions
using the assigned values of
$p_{1}$, $p_{2}$, $p_{3}$, $p_{4}$, and $p_{5}$.

Define Python variables `rxn_A` and `rxn_B` 
to store the forces developed at $A$ and $B$, respectively.

In [12]:
nodes = {
    'A': np.array([0.0, H]), 
    'B': np.array([0.0, 0.0]),
    'C': np.array([0.5*D, H*(1-0.5/3)]), 
    'D': np.array([D, 0.0])
}

u_AC = (nodes['C'] - nodes['A']) / np.linalg.norm(nodes['C'] - nodes['A'])
u_BC = (nodes['C'] - nodes['B']) / np.linalg.norm(nodes['C'] - nodes['B'])
u_BD = (nodes['D'] - nodes['B']) / np.linalg.norm(nodes['D'] - nodes['B'])

rxn_A = np.linalg.norm(-(phi_8[0] * u_AC))
rxn_B = np.linalg.norm(-(phi_8[1] * u_BC + phi_8[2] * u_BD))

print(f"Reaction at A: {rxn_A:.4f} kN")
print(f"Reaction at B: {rxn_B:.4f} kN")

Reaction at A: 20.4582 kN
Reaction at B: 31.9745 kN


## Instructions

Do not use any library or module other than those in the Imports section.

### Parameters

For each set,
the values shown are that of
$d$, $h$, $\theta$, $p_{1}$, $p_{2}$, $p_{3}$, $p_{4}$, and $p_{5}$,
respectively.

- Set 1: 1.5 ft, 4.5 ft, 21°, 5 kN, 7 kN, 6 kN, 8 kN, 1 kN
- Set 2: 2.0 ft, 3.0 ft, 17°, 9 kN, 6 kN, 5 kN, 1 kN, 7 kN
- Set 3: 1.0 ft, 6.0 ft, 15°, 1 kN, 9 kN, 8 kN, 7 kN, 5 kN

### Scoring

In each of Tasks 1 - 9,
obtaining a correct member force merits one (1) point.

In each Task but 7, 9, and 10,
- obtaining a correct determinant merits one (1) point,
  and
- obtaining a correct condition number merits one (1) point.

For Task 10,
obtaining a correct reaction force merits five (5) points.

Every violation of an instruction
means a deduction of one (1) point.

All in all, one may earn up to 124 points for this activity.

### Submission

Download this notebook file,
and save with a filename following the pattern
`ACT-08_<Group name>`,
where the group name is as listed in class.
For example, if you belong to the group SixIsEven,
then your notebook should be named `ACT-08_SixIsEven.ipynb`.
Submit your notebook via the classwork platform for this activity in Google Classroom.
Submissions beyond the deadline will not be considered.

Lastly, the use of AI tools to answer this exam is not prohibited,
but it is of ethical interest to disclose such use.
This is in line with the
[MSU Policy on the Fair and Ethical Use of AI and Its Applications](https://www.msumain.edu.ph/wp-content/uploads/2024/05/MSU-Policy-on-Ethical-use-of-AI-Policies.pdf).
As such, please include a brief statement (in a private comment to this classwork)
declaring which and how AI tools are used in your work.

*Last updated by Christian Cahig on 2025-11-25*